# Imports

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2

## Checking the first sample

In [ ]:
# Long running
y, x = load_data(train=True)
print(f"First sample: {x[0,:]}")
print(f"First sample label: {y[0]}")

## Tests

In [ ]:
# Define some test data
testing_y = np.array([1,1,2,2,4])
testing_x = np.array([[1],[2],[3],[4],[5]])
testing_w = np.array([-0.1, 0.7])

print(f"X:\n {testing_x}\n")
testing_sx, testing_mean_x, testing_std_x = standardize(testing_x) # Standardization
print(f"X normalized:\n {testing_sx}\n") 
testing_tx = add_x_bias(testing_sx) # Adding bias column to X
print(f"X with bias:\n {testing_tx}")


### MSE

In [ ]:
print("MSE TEST")
print(f"Got:{compute_mse(testing_y, testing_tx, testing_w)}")
print("Expected:" + str(4.71))

### MAE

In [ ]:
print("MAE TEST")
print(f"Got:{compute_mae(testing_y, testing_tx, testing_w)}")
print("Expected:" + str(2.1))

### MSE Gradient

In [ ]:
print("MSE Gradient TEST")
print(f"Got:{compute_mse_gradient(testing_y, testing_tx, testing_w)}")
print("Expected:" + str([-2.1, -0.29]))

### MAE Gradient

In [ ]:
print("MAE Gradient TEST")
print(f"Got:{compute_mae_gradient(testing_y, testing_tx, testing_w)}")
print("Expected:" + str([-1, -0]))

## Linear regression using gradient descent

In [ ]:
y, x = load_data(train=True) # Load train data
for i in range(np.shape(x)[1] + 1): # Iterate through x to replace -999 by the mean of its column (which is more natural)
    col_mean = np.nanmean(np.ma.MaskedArray(x[i, :], mask=(np.array(x[i, :]) == -999)))
    x[i] = [xi if xi != -999 else col_mean for xi in x[i, :]]
standardize(x)
tx = add_x_bias(x) # Add the bias term in x
print(np.shape(y))
print(np.shape(tx))

In [ ]:
w, loss = least_squares_SGD(y, tx, np.zeros(31), 100, 0.00000001) # Gamma is really small because w's tend to be really small compared to the gradient